In [232]:
import warnings

import pandas as pd
import polars as pl
import os
import pandas_datareader.data as web
import statsmodels.api as sm
import numpy as np
import yfinance as yf

from pathlib import Path
from IPython.display import display

warnings.filterwarnings('ignore')

In [2]:
root = Path(os.getcwd())

# Create Data

In [6]:
def get_load_data_large_dir():
    return root / 'data'

In [7]:
def get_load_data_parquet_dir():
    return root / 'parquet'

In [222]:
def create_fama():
    fama_data = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', start=2018)[0].rename(columns={'Mkt-RF': 'MARKET'})
    fama_data.index.names = ['date']
    fama_data = fama_data.resample('MS').first()
    fama_data.to_parquet(get_load_data_parquet_dir() / 'data_fama.parquet.brotli', compression='brotli')

In [223]:
create_fama()

In [224]:
fama = pd.read_parquet(get_load_data_parquet_dir() / 'data_fama.parquet.brotli')

In [225]:
fama

,MARKET,SMB,HML,RMW,CMA,RF
date,,,,,,
2018-01-01,0.85,0.30,-0.22,-0.50,0.31,0.006
2018-02-01,0.03,0.12,0.54,-0.53,0.00,0.006
2018-03-01,-1.19,0.97,-0.02,-0.24,0.05,0.005
2018-04-01,-2.29,-0.09,0.36,0.17,-0.07,0.007
2018-05-01,0.24,0.24,-0.54,-0.04,-0.96,0.006
...,...,...,...,...,...,...
2023-03-01,-0.42,0.40,0.62,0.20,0.61,0.016
2023-04-01,0.28,-0.19,0.78,0.51,0.70,0.018
2023-05-01,-0.02,0.17,-0.78,-0.50,0.18,0.016


In [180]:
def create_macro():
    medianCPI = pd.read_csv(get_load_data_large_dir() / 'macro' / 'medianCPI.csv')
    medianCPI.columns = ['date', 'medCPI']
    medianCPI = medianCPI.set_index(pd.to_datetime(medianCPI['date'])).drop('date', axis=1)
    medianCPI = medianCPI.shift(1)

    rGDP = pd.read_csv(get_load_data_large_dir() / 'macro' / 'realGDP.csv')
    rGDP.columns = ['date', 'rGDP']
    rGDP = rGDP.set_index(pd.to_datetime(rGDP['date'])).drop('date', axis=1)

    rIR = pd.read_csv(get_load_data_large_dir() / 'macro' / 'realInterestRate.csv')
    rIR.columns = ['date', 'rIR']
    rIR = rIR.set_index(pd.to_datetime(rIR['date'])).drop('date', axis=1)
    rIR = rIR.shift(1)

    UR = pd.read_csv(get_load_data_large_dir() / 'macro' / 'unemploymentRate.csv')
    UR.columns = ['date', 'UR']
    UR = UR.set_index(pd.to_datetime(UR['date'])).drop('date', axis=1)
    UR = UR.shift(1)

    TB = pd.read_csv(get_load_data_large_dir() / 'macro' / 'TB.csv')
    TB.columns = ['date', 'TB']
    TB = TB.set_index(pd.to_datetime(TB['date'])).drop('date', axis=1)
    TB = TB.shift(1)
    PPI = pd.read_csv(get_load_data_large_dir() / 'macro' / 'PPI.csv')
    PPI.columns = ['date', 'PPI']
    PPI = PPI.set_index(pd.to_datetime(PPI['date'])).drop('date', axis=1)
    PPI = PPI.shift(1)
    
    retailSales = pd.read_csv(get_load_data_large_dir() / 'macro' / 'retailSales.csv')
    retailSales.columns = ['date', 'retailSales']
    retailSales = retailSales.set_index(pd.to_datetime(retailSales['date'])).drop('date', axis=1)
    retailSales = retailSales.shift(1)
    
    indProdIndex = pd.read_csv(get_load_data_large_dir() / 'macro' / 'indProdIndex.csv')
    indProdIndex.columns = ['date', 'indProdIndex']
    indProdIndex = indProdIndex.set_index(pd.to_datetime(indProdIndex['date'])).drop('date', axis=1)
    indProdIndex = indProdIndex.shift(1)

    realDispoIncome = pd.read_csv(get_load_data_large_dir() / 'macro' / 'realDispoIncome.csv')
    realDispoIncome.columns = ['date', 'realDispoIncome']
    realDispoIncome = realDispoIncome.set_index(pd.to_datetime(realDispoIncome['date'])).drop('date', axis=1)
    realDispoIncome = realDispoIncome.shift(1)
    
    def pctChange(data, name):
        data.replace('.', np.nan, inplace=True)
        data = data.astype(float)
        data[f'{name}_pct']=data[f'{name}'].pct_change()
        return data
    
    medianCPI = pctChange(medianCPI, 'medCPI')
    rGDP = pctChange(rGDP, 'rGDP')
    rIR = pctChange(rIR, 'rIR')
    UR = pctChange(UR, 'UR')
    TB = pctChange(TB, 'TB')
    PPI = pctChange(PPI, 'PPI')
    retailSales = pctChange(retailSales, 'retailSales')
    indProdIndex = pctChange(indProdIndex, 'indProdIndex')
    realDispoIncome = pctChange(realDispoIncome, 'realDispoIncome')
    macro = (pd.merge(medianCPI, rGDP, left_index=True, right_index=True, how='left')
                 .merge(rIR, left_index=True, right_index=True, how='left')
                 .merge(UR, left_index=True, right_index=True, how='left')
                 .merge(TB, left_index=True, right_index=True, how='left')
                 .merge(PPI, left_index=True, right_index=True, how='left')
                 .merge(retailSales, left_index=True, right_index=True, how='left')
                 .merge(indProdIndex
                        , left_index=True, right_index=True, how='left')
                 .merge(realDispoIncome, left_index=True, right_index=True, how='left'))
    
    factor_macro = macro[['medCPI_pct', 'rGDP_pct', 'rIR_pct', 'UR_pct', 'TB_pct', 'PPI_pct', 'retailSales_pct', 'indProdIndex_pct', 'realDispoIncome_pct']]
    factor_macro = factor_macro.replace([np.inf, -np.inf], np.nan)
    factor_macro = factor_macro.loc["2018-01-01":"2023-07-01"]
    factor_macro.to_parquet(get_load_data_parquet_dir() / 'data_macro.parquet.brotli', compression='brotli')

In [181]:
create_macro()

In [182]:
macro = pd.read_parquet(get_load_data_parquet_dir() / 'data_macro.parquet.brotli')

In [183]:
macro

,medCPI_pct,rGDP_pct,rIR_pct,UR_pct,TB_pct,PPI_pct,retailSales_pct,indProdIndex_pct,realDispoIncome_pct
date,,,,,,,,,
2018-01-01,0.221185,0.006927,-0.238159,-0.023810,0.023580,0.002042,0.006047,0.001951,0.001458
2018-02-01,0.410811,NaN,0.524804,-0.024390,0.011540,0.008151,-0.005748,-0.000625,0.006598
2018-03-01,-0.517199,NaN,0.245690,0.025000,0.036561,0.007074,0.006285,0.003666,0.001729
2018-04-01,-0.025520,0.006973,-0.054143,-0.024390,-0.108315,0.000000,-0.001990,0.004702,0.003078
2018-05-01,0.494474,NaN,-0.014746,0.000000,0.060242,0.005018,0.004484,0.010785,0.002373
...,...,...,...,...,...,...,...,...,...
2023-02-01,0.155101,NaN,0.131318,-0.028571,-0.017006,0.009035,0.024928,0.010474,0.016662
2023-03-01,-0.032177,NaN,-0.203520,0.058824,0.000827,-0.005987,-0.004214,0.000255,0.002619
2023-04-01,-0.377317,0.005982,0.450242,-0.027778,-0.137206,-0.006213,-0.010635,0.000868,0.003429


In [126]:
def create_etf():
    etf_tickers = pl.scan_csv(get_load_data_large_dir() / 'tickers_etf.csv').collect(streaming=True).to_series().to_list()
    start_date = "2018-01-01"
    end_date = "2023-07-01"
    etf_data = yf.download(etf_tickers, start=start_date, end=end_date, interval='1mo')
    etf_data = etf_data.stack().swaplevel().sort_index()
    etf_data.index.names = ['ticker', 'date']
    etf_data = etf_data.astype(float)
    
    # Calculate returns of each ticker and rename each return column to ticker
    ret = etf_data.groupby('ticker')['Close'].transform(lambda x: x.pct_change())
    ret_df = ret.unstack(level='ticker')
    dates = etf_data.reset_index('ticker').drop(
        ['ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], axis=1)
    dates = dates.loc[~dates.index.duplicated(keep='first')].sort_index()
    etf_data = pd.concat([dates, ret_df], axis=1)
    etf_data.to_parquet(get_load_data_parquet_dir() / 'data_etf.parquet.brotli', compression='brotli')

In [127]:
create_etf()

[*********************100%%**********************]  10 of 10 completed


In [128]:
etf = pd.read_parquet(get_load_data_parquet_dir() / 'data_etf.parquet.brotli')

In [129]:
etf

,IVV,IWR,QQQ,SPYG,TIP,TLT,VNQ,XLI,XLP,XLV
date,,,,,,,,,,
2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-01,-0.038043,-0.040680,-0.012928,-0.021204,-0.010521,-0.032429,-0.076778,-0.038630,-0.076271,-0.044949
2018-03-01,-0.029158,-0.002946,-0.042342,-0.031773,0.010097,0.026526,0.028903,-0.030789,-0.014604,-0.032565
2018-04-01,0.003542,-0.001792,0.005058,0.002088,-0.003892,-0.022970,0.008215,-0.027864,-0.041421,0.010565
2018-05-01,0.024107,0.022711,0.056729,0.043465,-0.001154,0.017800,0.036799,0.030739,-0.015659,0.001824
...,...,...,...,...,...,...,...,...,...,...
2023-02-01,-0.025324,-0.025188,-0.003598,-0.019436,-0.013530,-0.050947,-0.058643,-0.008640,-0.023190,-0.046416
2023-03-01,0.032942,-0.018115,0.093235,0.055079,0.028643,0.045817,-0.031265,0.002080,0.037207,0.018007
2023-04-01,0.016007,-0.005721,0.005079,0.014270,-0.000635,0.000846,0.003131,-0.011662,0.036541,0.031438


In [246]:
def create_emr():
    start_date = "2018-01-01"
    end_date = "2023-07-01"
    emr_data = yf.download('EMR', start=start_date, end=end_date, interval = '1mo')
    emr_data.index.name = 'date'
    emr_data['ticker'] = 'EMR'
    emr_data = emr_data.reset_index().set_index(['ticker', 'date'])
    emr_data['RET_01'] = emr_data.Close.pct_change()
    emr_data[['RET_01']].to_parquet(get_load_data_parquet_dir() / 'data_emr.parquet.brotli')

In [247]:
create_emr()

[*********************100%%**********************]  1 of 1 completed


In [248]:
emr_data = pd.read_parquet(get_load_data_parquet_dir() / 'data_emr.parquet.brotli')

In [249]:
emr_data

RET_01
ticker date                
EMR    2018-01-01       NaN
       2018-02-01 -0.016198
       2018-03-01 -0.038840
       2018-04-01 -0.027672
       2018-05-01  0.066707
...                     ...
       2023-02-01 -0.083241
       2023-03-01  0.053561
       2023-04-01 -0.044526
       2023-05-01 -0.067019
       2023-06-01  0.163620

[66 rows x 1 columns]

# Cost of Equity

In [250]:
factor_data = pd.concat([macro, etf, fama], axis=1)
factor_data = factor_data.loc['2018-01':'2023-06-01']
factor_data = factor_data.fillna(0)
factor_col = factor_data.columns[:-1]

In [252]:
# Linear Regression
betas = []
ret = 'RET_01'

model_data = emr_data[[ret]].merge(factor_data, on='date').dropna()
model_data[ret] -= model_data.RF

# Fit OLS regression
ols_model = sm.OLS(endog=model_data[ret], exog=sm.add_constant(model_data[factor_col])).fit()

# Extract coefficients
alpha = ols_model.params['const']
beta_coef = ols_model.params.drop('const')
factor_ret = model_data[factor_col]

# Compute predictions
predictions = factor_ret @ beta_coef + alpha

# Construct result dataframe
result = pd.DataFrame({'ALPHA': alpha, **beta_coef, 'PRED': predictions}).assign(ticker='EMR').set_index('ticker', append=True).swaplevel()

betas.append(result)

final_df = pd.concat(betas).rename(columns=lambda x: f'{x}_EMR')

In [253]:
final_df

ALPHA_EMR  medCPI_pct_EMR  rGDP_pct_EMR  rIR_pct_EMR  \
ticker date                                                               
EMR    2018-02-01  -0.006797        0.007338     -1.244948    -0.001605   
       2018-03-01  -0.006797        0.007338     -1.244948    -0.001605   
       2018-04-01  -0.006797        0.007338     -1.244948    -0.001605   
       2018-05-01  -0.006797        0.007338     -1.244948    -0.001605   
       2018-06-01  -0.006797        0.007338     -1.244948    -0.001605   
...                      ...             ...           ...          ...   
       2023-02-01  -0.006797        0.007338     -1.244948    -0.001605   
       2023-03-01  -0.006797        0.007338     -1.244948    -0.001605   
       2023-04-01  -0.006797        0.007338     -1.244948    -0.001605   
       2023-05-01  -0.006797        0.007338     -1.244948    -0.001605   
       2023-06-01  -0.006797        0.007338     -1.244948    -0.001605   

                   UR_pct_EMR  TB_pct_EMR  PPI_pct_EMR  retailSales_pct_EMR  \
ticker date                                                                   
EMR    2018-02-01    0.094289   -0.046306     0.648141             -0.18163   
       2018-03-01    0.094289   -0.046306     0.648141             -0.18163   
       2018-04-01    0.094289   -0.046306     0.648141             -0.18163   
       2018-05-01    0.094289   -0.046306     0.648141             -0.18163   
       2018-06-01    0.094289   -0.046306     0.648141             -0.18163   
...                       ...         ...          ...                  ...   
       2023-02-01    0.094289   -0.046306     0.648141             -0.18163   
       2023-03-01    0.094289   -0.046306     0.648141             -0.18163   
       2023-04-01    0.094289   -0.046306     0.648141             -0.18163   
       2023-05-01    0.094289   -0.046306     0.648141             -0.18163   
       2023-06-01    0.094289   -0.046306     0.648141             -0.18163   

                   indProdIndex_pct_EMR  realDispoIncome_pct_EMR  ...  \
ticker date                                                       ...   
EMR    2018-02-01              1.418692                -0.125972  ...   
       2018-03-01              1.418692                -0.125972  ...   
       2018-04-01              1.418692                -0.125972  ...   
       2018-05-01              1.418692                -0.125972  ...   
       2018-06-01              1.418692                -0.125972  ...   
...                                 ...                      ...  ...   
       2023-02-01              1.418692                -0.125972  ...   
       2023-03-01              1.418692                -0.125972  ...   
       2023-04-01              1.418692                -0.125972  ...   
       2023-05-01              1.418692                -0.125972  ...   
       2023-06-01              1.418692                -0.125972  ...   

                    VNQ_EMR   XLI_EMR   XLP_EMR  XLV_EMR  MARKET_EMR  \
ticker date                                                            
EMR    2018-02-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   
       2018-03-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   
       2018-04-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   
       2018-05-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   
       2018-06-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   
...                     ...       ...       ...      ...         ...   
       2023-02-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   
       2023-03-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   
       2023-04-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   
       2023-05-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   
       2023-06-01 -0.058179  0.975035 -0.195555  0.31963   -0.002114   

                    SMB_EMR   HML_EMR   RMW_EMR   CMA_EMR  PRED_EMR  
ticker date                                               

In [261]:
cost_equity = final_df['PRED_EMR'] + fama.loc['2018-01':'2023-06-01']['RF']

In [262]:
cost_equity = cost_equity.to_frame('cost_equity')

In [265]:
# Resample the data for yearly by compounding monthly returns
def compound_returns(x):
    return (x + 1).prod() - 1

In [267]:
cost_equity_emr = cost_equity.loc['EMR']
cost_equity_yearly = cost_equity_emr.resample('A').apply(compound_returns)

In [268]:
cost_equity_yearly

,cost_equity
date,
2018-12-31,-0.195803
2019-12-31,0.332359
2020-12-31,0.016249
2021-12-31,0.163586
2022-12-31,-0.036957
2023-12-31,0.055900


In [273]:
ce = 0.0559

# Cost of Debt

Using last fiscal year end Interest Expense divided by the latest two-year average debt to get the simplified cost of debt.
As of Sep. 2022, Emerson Electric Co's interest expense (positive number) was 228 million. Its total Book Value of Debt (D) is 8900 million.
Cost of Debt => 228 / 8900 = 0.02561797752 or 2.56%.

In [276]:
cd = 228 / 8900

# WACC

##### WACC = (Cost of Equity * % Equity) + (Cost of Debt * % Debt)(1-Tax Rate) + (Preferred Shares * % Preferred Shares)

Tax rate = Income Tax Expense (Income) / Income Before Tax (EBT - Income)

Total Cap = Total Equity (Balance) + Total Debt (Balance)

Preferred = Number of Preferred Shares (10-K)

% Equity = Total Equity / Total Cap

% Debt = Total Debt / Total Cap

% Preferred = Number of Preferred Shares / Number of Outstanding Shares

In [274]:
tax = 846.0 / 4078.0

In [290]:
preferred = 0

In [288]:
outstanding = 591.4

In [279]:
total_equity = 26413.0

In [278]:
total_debt = 8815.0

In [311]:
total_cap = total_equity + total_debt
total_cap

35228.0

In [312]:
ep = total_equity/total_cap
ep

0.7497729079141592

In [313]:
ed = total_debt/total_cap
ed

0.2502270920858408

In [314]:
wacc = (ce*ep + cd*ed*(1-tax))*100

In [315]:
print(f"EMR's WACC is: {round(wacc, 1)}%")

EMR's WACC is: 4.7%
